# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [19]:
import main


In [20]:
from sqlalchemy import create_engine
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')


In [21]:
# ZADANIE 1
# Znajdź listę wszystkich filmów o tej samej długości.

df =  pd.read_sql("""select length, title
                     from film
                     order by length
                  """,con=connection)
print(df)

# wykorzystuje funkcje length w celu wyswietlenia wsystkich filmow w tej samej dlugosci

     length                title
0        46  Ridgemont Submarine
1        46            Iron Moon
2        46         Alien Center
3        46        Kwai Homeward
4        46     Labyrinth League
..      ...                  ...
995     185       Darn Forrester
996     185    Sweet Brotherhood
997     185   Soldiers Evolution
998     185         Worst Banger
999     185       Control Anthem

[1000 rows x 2 columns]


C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/2888089081.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql("""select length, title


In [22]:
# ZADANIE 2
# Znajdź wszystkich klientów mieszkających w tym samym mieście.

df =  pd.read_sql("""select count(customer.last_name), city.city
                     from customer 
                     inner join address on customer.address_id = address.address_id
                     inner join city on address.city_id = city.city_id
                     group by city.city
                     order by count(customer.last_name) asc
                  """,con=connection)
print(df)

df = pd.read_sql("""select city.city, customer.first_name, customer.last_name
                    from customer 
                    inner join address on customer.address_id = address.address_id 
                    inner join city on address.city_id = city.city_id
                    where city.city in ('London', 'Aurora')
                    order by city desc
                 """,con=connection)
print(df)

# w tym zadaniu najpierw pogrupowalem wszystkich klientow ktorzy potencjalnie mogliby mieszkac w tym samym miescie
# a nastepnie wyszukalem ze sa to osoby z Londynu oraz z Aurora 

C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/3450202584.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql("""select count(customer.last_name), city.city


     count        city
0        1    Pemalang
1        1      Taguig
2        1       Tokat
3        1     Atlixco
4        1    Mukateve
..     ...         ...
592      1      Warren
593      1   Pingxiang
594      1  Greensboro
595      2      London
596      2      Aurora

[597 rows x 2 columns]
     city first_name last_name
0  London     Mattie   Hoffman
1  London      Cecil     Vines
2  Aurora      Scott   Shelley
3  Aurora    Clinton    Buford


C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/3450202584.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select city.city, customer.first_name, customer.last_name


In [23]:
# ZADANIE 3 
# Oblicz średni koszt wypożyczenia wszystkich filmów.

df = pd.read_sql("""select avg(payment.amount)
                    from payment 
                    inner join rental on payment.rental_id = rental.rental_id
                    inner join inventory on rental.inventory_id = inventory.inventory_id 
                    inner join film on inventory.film_id = film.film_id
                """,con=connection)
print(df)

# skorzystalem z funkcji avg, aby wyswietlic sredni koszt wypozyczenia filmow

        avg
0  4.200606


C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/2326710971.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select avg(payment.amount)


In [24]:
# ZADANIE 4
# Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

df = pd.read_sql("""select category.name, count(film.title)
                    from film 
                    inner join film_category on film.film_id = film_category.film_id
                    inner join category on film_category.category_id = category.category_id  
                    group by category.name
                """,con=connection)
print(df)

# w tym zadaniu pogrupowalem wszystkie kategorie filmow oraz obliczylem ile filmow nalezy do danej kategorii

C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/1537116976.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select category.name, count(film.title)


           name  count
0        Sports     74
1      Classics     57
2           New     63
3        Family     69
4        Comedy     58
5     Animation     66
6        Travel     57
7         Music     51
8         Drama     62
9        Horror     56
10       Sci-Fi     61
11        Games     61
12  Documentary     68
13      Foreign     73
14       Action     64
15     Children     60


In [25]:
# ZADANIE 5
# Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

df = pd.read_sql("""select count(customer.last_name), country.country
                    from customer 
                    inner join address on customer.address_id = address.address_id 
                    inner join city on address.city_id = city.city_id 
                    inner join country on city.country_id = country.country_id
                    group by country.country
                    order by country.country asc
                 """,con=connection)
print(df)

# w tym zadaniu z wykorzystaniem funkcji count wyliczylem wszystkich klientow a nastepnie pogrupowalem ich wedlug kraju

C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/2156270871.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select count(customer.last_name), country.country


     count               country
0        1           Afghanistan
1        3               Algeria
2        1        American Samoa
3        2                Angola
4        1              Anguilla
..     ...                   ...
103      6               Vietnam
104      1  Virgin Islands, U.S.
105      4                 Yemen
106      2            Yugoslavia
107      1                Zambia

[108 rows x 2 columns]


In [33]:
# ZADANIE 6
# Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów. 

df = pd.read_sql("""select address.address, count(customer.last_name)
                    from customer
                    inner join rental on customer.customer_id = rental.customer_id
                    inner join staff on rental.staff_id = staff.staff_id
                    inner join store on staff.store_id = store.store_id
                    inner join address on store.address_id = address.address_id
                    group by address.address 
                    order by count(customer.last_name) > 100 and count(customer.last_name) < 300 
                 """,con=connection)
print(df)

              address  count
0  28 MySQL Boulevard   8004
1   47 MySakila Drive   8040


C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/2836171324.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select address.address, count(customer.last_name)


In [27]:
# ZADANIE 7
# Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

df = pd.read_sql("""select customer.last_name, count(film.length)
                    from customer
                    inner join rental on customer.customer_id = rental.customer_id
                    inner join inventory on rental.inventory_id = inventory.inventory_id
                    inner join film on inventory.film_id = film.film_id
                    group by customer.last_name 
                    having count(film.length) > 200
                """,con=connection)
print(df)

# z moich obliczen wyszlo ze zadnen klient nie ogladal zadnego filmu, ktory trwal ponad 200 godzin

C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/2744321139.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select customer.last_name, count(film.length)


Empty DataFrame
Columns: [last_name, count]
Index: []


In [28]:
# ZADANIE 8
# Oblicz średnią wartość wypożyczenia filmu

df = pd.read_sql("""select avg(rental_rate) 
                    from film 
                 """,con=connection)
print(df)

# wystarczylo wykorzysac funkcje avg w kolumnie "film", aby wyliczyc srednia wartosc wypozyczenia filmu

C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/1296869088.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select avg(rental_rate)


    avg
0  2.98


In [29]:
# ZADANIE 9
# Oblicz średnią wartość długości filmu we wszystkich kategoriach.

df = pd.read_sql("""select category.name, avg(film.length) 
                    from category 
                    inner join film_category on category.category_id = film_category.category_id
                    inner join film on film.film_id = film_category.film_id
                    group by category.name
                    order by category.name desc
                 """,con=connection)
print(df)

# wykorzystuje funkcje avg i przeskakuje po odpowiednich kolumnach w celu wyliczenia sredniej wartosci dlugosci filmu kazdej kategorii

           name         avg
0        Travel  113.315789
1        Sports  128.202703
2        Sci-Fi  108.196721
3           New  111.126984
4         Music  113.647059
5        Horror  112.482143
6         Games  127.836066
7       Foreign  121.698630
8        Family  114.782609
9         Drama  120.838710
10  Documentary  108.750000
11       Comedy  115.827586
12     Classics  111.666667
13     Children  109.800000
14    Animation  111.015152
15       Action  111.609375


C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/3910269141.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select category.name, avg(film.length)


In [30]:
# ZADANIE 10
# Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

df = pd.read_sql("""select film.title, film.length 
                    from category 
                    inner join film_category on category.category_id = film_category.category_id
                    inner join film on film.film_id = film_category.film_id
                    order by film.length desc
                 """,con=connection)
print(df)

# to zadnie bylo bardzo podobne do 9, zmienilem tylko wartosci, ktore potrzebowalem do wyprintowania:)
# przeskakiwanie po katalogach takie samo jak w zadaniu wyzej

                   title  length
0              Home Pity     185
1          Muscle Bright     185
2         Darn Forrester     185
3      Sweet Brotherhood     185
4           Worst Banger     185
..                   ...     ...
995            Iron Moon      46
996         Alien Center      46
997        Kwai Homeward      46
998  Ridgemont Submarine      46
999     Labyrinth League      46

[1000 rows x 2 columns]


C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/1813430194.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select film.title, film.length


In [31]:
# ZADANIE 11
# Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

df = pd.read_sql("""select category.name as film_category, max(film.length) as max_length
                    from category 
                    inner join film_category on category.category_id = film_category.category_id
                    inner join film on film.film_id = film_category.film_id
                    group by category.name
                    order by max(film.length)
                 """,con=connection)
print(df)



   film_category  max_length
0       Children         178
1          Drama         181
2         Horror         181
3    Documentary         183
4            New         183
5       Classics         184
6         Sports         184
7         Family         184
8        Foreign         184
9         Action         185
10        Sci-Fi         185
11         Games         185
12     Animation         185
13        Travel         185
14         Music         185
15        Comedy         185


C:\Users\Acer\AppData\Local\Temp/ipykernel_26576/4164515833.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("""select category.name as film_category, max(film.length) as max_length
